In [15]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
from datetime import datetime
from pprint import pprint
from bson import ObjectId

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [5]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/4_consistent/omero_museum"

In [6]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room startDate ticketIds workshopTitle
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donationState donatorId isOriginal locationName materials period sellerId size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[employees]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[limited_events]: _id artist artworkTitles authorIds capacity description endDate roomName startDate theme ticketIds type
[messages]: _id content customerId deliveryStatus length sentDate type uri
[rooms]: _id floor
[suppliers]: _id email iban isMuseum isState name phoneNum surveys
[visitors]: _id birthDate chat comments donationIds email gender impairment isCustomer land name phoneNum saleIds surname surveys 

### CR1: _Create a new comment to a specific artwork_

In [8]:
db.artworks.update_one(
    {"_id": "Mona Lisa"},
    {
        "$push": {
            "comments_star_5": { # N-star comment goes to comments_star_N array
                "rating": 5, # N-star rating
                "date": datetime.now(),
                "message": "Capolavoro assoluto, emozionante da vedere dal vivo!"
            }
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR2: _Create a new survey compilation_

In [9]:
db.visitors.update_one(
    {"_id": 0},
    {
        "$push": {
            "surveys": {
                "accompanying_persons_visit": "amici",
                "date_of_compilation": datetime(2025, 9, 5, 15, 30),
                "reason_for_visit": "Interesse per l'arte moderna",
                "number_of_visits": 2,
                "return": "sì",
                "type_of_visit": "visita guidata",
                "title_of_studies": "Diploma",
                "evaluation_of_experience": 9,
                "evaluation_of_facility": 8,
                "evaluation_of_visit": 10
            }
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR3: _Create a new ticket for a visitor_

In [10]:
db.visitors.update_one(
    {"_id": 0},
    {
        "$push": {
            "tickets": {
                "date": datetime(2025, 9, 6, 11, 0),
                "price": 18.0,
                "check_in": False
            }
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR4: _Create a new message_

In [13]:
visitor_id = 0

new_message = {
    "customerId": visitor_id,
    "sentDate": datetime(2025, 9, 5, 16, 0),
    "type": "text",
    "content": "Benvenuto al Museo Omero!",
    "deliveryStatus": "sent",
    "length": 24,
    "uri": None
}

visitor = db.visitors.find_one({"_id": visitor_id})
chat = visitor.get("chat", {})
new_msgs = chat.get("new_messages", [])
old_msgs = chat.get("old_messages", [])

new_msgs.append(new_message)

# If there are more than 10 new messages, move the oldest to old messages
if len(new_msgs) > 10:
    oldest = new_msgs.pop(0)
    old_msgs.insert(0, oldest)

db.visitors.update_one(
    {"_id": visitor_id},
    {
        "$set": {
            "chat.new_messages": new_msgs,
            "chat.old_messages": old_msgs
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR5: _Create a new trade_

In [14]:
db.artworks.update_one(
    {"_id": "Mona Lisa"},
    {
        "$set": {
            "trade": {
                "date_start": datetime(2025, 9, 10, 10, 0),
                "date_end": datetime(2025, 9, 15, 18, 0),
                "win_price": 12000,
                "winner_id": 20,  # Link to the visitor
                "price_max": 15000,
                "price_min": 8000,
                "price_start": 5000,
                "min_increase": 200,
                "is_auction": True
            }
        }
    }
)


db.visitors.update_one(
    {"_id": 20},
    {"$addToSet": {"tradeIds": "Mona Lisa"}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### CR6: _Create a new visitor_

In [ ]:
last = db.visitors.find_one(sort=[("_id", -1)])
next_id = (last["_id"] + 1) if last else 1

db.visitors.insert_one({
    "_id": next_id,
    "isCustomer": True,
    "birthDate": datetime(1990, 5, 15, 2, 0),
    "gender": "male",
    "land": "Italy",
    "tickets": [
        {
            "date": datetime(2024, 7, 1, 10, 0),
            "price": 15.0,
            "check_in": True
        }
    ],
    "surname": "Rosa",
    "name": "Maria",
    "phoneNum": "+390123456789",
    "email": "maria.rosa@.com",
    "comments": [],
    "donationIds": [],
    "saleIds": [],
    "tradeIds": []
})


InsertOneResult(53, acknowledged=True)

### CR7: _Create a new activity_

In [ ]:
last = db.activities.find_one(sort=[("_id", -1)])
next_id = (last["_id"] + 1) if last else 1

db.activities.insert_one({
    "_id": next_id,
    "room": "Rinascimentale",  
    "duration": 90,
    "enrolled": 5,
    "startDate": datetime(2025, 9, 4, 15, 0),
    "capacity": 30,
    "ticketIds": [34, 46, 9, 7, 12],
    "workshopTitle": "Pittura su tela"  
})


InsertOneResult(25, acknowledged=True)

### CR8: _Create a new artwork_

In [ ]:
db.artworks.insert_one({
    "_id": "La Notte Stellata",
    "date": datetime(1889, 6, 1),
    "type": "painting",
    "description": "Opera celebre di Van Gogh",
    "isOriginal": True,
    "size": 73,
    "period": "Post-Impressionismo",
    "sellerId": 9,     
    "donatorId": 13,    
    "donationState": "refused",
    "locationName": "Parigi",
    "authorIds": ["68b572f8845fcdbabc1c8906"],  # authors
    "tecniques": ["mosaic"],
    "materials": ["Glass", "wood"],
    "comments_star_1": [],
    "comments_star_2": [],
    "comments_star_3": [
        {
            "rating": 3,
            "date": datetime(2025, 8, 12, 10, 30),
            "message": "Un buon lavoro, ma si può migliorare"
        }
    ],    
    "comments_star_4": [],
    "comments_star_5": [],
    "trade": {
        "date_start": datetime(2025, 8, 24, 22, 52, 10),
        "date_end": datetime(2025, 8, 30, 17, 42, 7),   
        "win_price": 4800,
        "winner_id": 3,
        "price_max": 5000,
        "price_min": 2900,
        "price_start": 2500,
        "min_increase": 50,
        "is_auction": False
    }
})

InsertOneResult('La Notte Stellata', acknowledged=True)

### CR9: _Create a new limited event_

In [ ]:
db.limited_events.insert_one({
    "_id": "The Future of Mosaic",
    "capacity": 40,
    "type": "exhibition",
    "startDate": datetime(2025, 10, 1, 10, 0),
    "endDate": datetime(2025, 10, 1, 18, 0),
    "artist": 2,  # artista o id
    "theme": "Innovation",
    "description": "Mostra temporanea di opere moderne",
    "roomName": "Contemporaneo",  # room id
    "authorIds": [],
    "artworkTitles": ["The Scream", "Guernica"],
    "ticketIds": [42, 51, 8, 6, 7]
})

InsertOneResult('The Future of Mosaic', acknowledged=True)

### CR10: _Create a new employee_ (correction json first)

### CR11: _Create a new shift_ (correction json first)

### CR12: _Create a new author_

In [16]:
db.authors.insert_one({
    "_id": ObjectId(),
    "surname": "Rossi",
    "birth_date": datetime(1975, 4, 12),
    "home_town": "Milano",
    "name": "Giovanni",
    "gender": "M"
})

InsertOneResult(ObjectId('68bab0df278cb5df47bf676b'), acknowledged=True)

### CR13: _Create a new workshop_

In [ ]:
db.workshops.insert_one({
    "_id": "Pittura su tela",
    "duration": datetime(1970, 1, 1, 2, 0),         
    "price_class": 80.00,        
    "price_person": 25.00,      
    "type": "Design"  
})

InsertOneResult('Pittura su tela', acknowledged=True)

### CR14: _Create a new supplier_

In [ ]:
last = db.suppliers.find_one(sort=[("_id", -1)])
next_id = (last["_id"] + 1) if last else 1

db.suppliers.insert_one({
    "_id": next_id,
    "name": "Maurizio Conte",
    "iban": "IT60X0542811101000000123456",
    "isState": True,
    "isMuseum": False,
    "email": "conte_mauto@outlook.it",
    "phoneNum": "+390276543210",
    "surveys": [
        {
            "accompanying_persons_visit": "solo",
            "date_of_compilation": datetime(2025, 8, 20, 14, 30),
            "reason_for_visit": "dovevo aspettare 1 ora il treno",
            "number_of_visits": 0,
            "return": "probabilmente no",
            "type_of_visit": "visita libera",
            "title_of_studies": "Laurea",
            "evaluation_of_experience": 7,
            "evaluation_of_facility": 4,
            "evaluation_of_visit": 6
        }
    ]
})


InsertOneResult(25, acknowledged=True)